# Tuan Nguyen's Project
## Data Engineering Capstone Project

## Project Summary

- The purpose of the data engineering capstone project is to give student a chance to combine what I learned throughout the program. This project will be an important part of my portfolio that will help you achieve my data engineering-related career goals.

## The project follows the follow steps:
* **Step 1**: Scope the Project and Gather Data
* **Step 2**: Explore and Assess the Data
* **Step 3**: Define the Data Model
* **Step 4**: Run ETL to Model the Data
* **Step 5**: Complete Project Write Up

---

# Step 1: Scope the Project and Gather Data

## Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

- We will gather data, then explore it to see what is inside of them. 
- We will use the Udacity provided project datasets. 
- The end solutions should be an OLAP model which can be used by Data Analysts/Data Scientists to work with. 
- The tools will be used are:
    - Spark
    - https://dbdiagram.io/ - A database diagram tool we use to draw the relational model

# Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

## Describe datasets

In the Udacity provided project, we'll work with four datasets to complete the project. 

The main dataset will include data on immigration to the United States, and supplementary datasets will include data on airport codes, U.S. city demographics, and temperature data. 

We're also welcome to enrich the project with additional data if we'd like to set your project apart.

- **I94 Immigration Data**: This data comes from the US National Tourism and Trade Office. A data dictionary is included in the workspace. This is where the data comes from. There's a sample file so you can take a look at the data in csv format before reading it all in. You do not have to use the entire dataset, just use what you need to accomplish the goal you set at the beginning of the project.
- **World Temperature Data**: This dataset came from Kaggle. You can read more about it here.
- **U.S. City Demographic Data**: This data comes from OpenSoft. You can read more about it here.
- **Airport Code Table**: This is a simple table of airport codes and corresponding cities. It comes from here.

## Gather data

In [1]:
# Do all imports and installs here
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import substring, translate
from pyspark.sql.functions import year, month, dayofmonth
from pyspark.sql.functions import trim

*First, we tried to use Pandas DataFrame but it was kind of slow*

In [2]:
# Read in the data here
# fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
# df = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")
# df.head()

In [3]:
# fname = '../../data2/GlobalLandTemperaturesByCity.csv'
# df = pd.read_csv(fname)
# df.head()

*Then we switched to Spark, which can work with high computational services*

### Read I94 Immigration SAS Dataset

In [4]:
spark = SparkSession \
    .builder \
    .config("spark.jars.repositories", "https://repos.spark-packages.org/") \
    .config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11") \
    .enableHiveSupport() \
    .getOrCreate()

sas_df = spark \
    .read.format('com.github.saurfang.sas.spark') \
    .load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat') \

# Load full data
# sas_df = spark \
#     .read.format('com.github.saurfang.sas.spark') \
#     .load('../../data/18-83510-I94-Data-2016/') \

# write to parquet
# sas_df.write.mode('overwrite').parquet("sas_data")

sas_df.printSchema()
sas_df.count()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

3096313

### Read World Temperature Dataset

In [5]:
global_temp_df = spark.read \
    .option("header", True) \
    .option("inferSchema", True) \
    .csv('../../data2/GlobalLandTemperaturesByCity.csv')

global_temp_df.printSchema()
global_temp_df.count()

root
 |-- dt: timestamp (nullable = true)
 |-- AverageTemperature: double (nullable = true)
 |-- AverageTemperatureUncertainty: double (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)



8599212

### Read US Cities Demographic Dataset

In [6]:
us_cities_df = spark.read \
    .option("delimiter", ";") \
    .option("header", True) \
    .option("inferSchema", True) \
    .csv('us-cities-demographics.csv')

us_cities_df.printSchema()
us_cities_df.count()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: double (nullable = true)
 |-- Male Population: integer (nullable = true)
 |-- Female Population: integer (nullable = true)
 |-- Total Population: integer (nullable = true)
 |-- Number of Veterans: integer (nullable = true)
 |-- Foreign-born: integer (nullable = true)
 |-- Average Household Size: double (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: integer (nullable = true)



2891

### Read Airport Code Dataset

In [7]:
airport_code_df = spark.read \
    .option("delimiter", ",") \
    .option("header", True) \
    .option("inferSchema", True) \
    .csv('airport-codes_csv.csv')

airport_code_df.printSchema()
airport_code_df.count()

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: integer (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)



55075

---

# Step 2: Explore and Assess the Data
## Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

### Explore I94 data

In [8]:
sas_df.select(sas_df.columns[:15]).limit(5).toPandas()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,None,NaN,37.0,2.0,1.0,None,None
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,25.0,3.0,1.0,20130811,SEO
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,55.0,2.0,1.0,20160401,None
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,28.0,2.0,1.0,20160401,None
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,4.0,2.0,1.0,20160401,None


In [9]:
sas_df.select(sas_df.columns[15:]).limit(5).toPandas()

,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,None,T,None,U,None,1979.0,10282016,None,None,None,1.897628e+09,None,B2
1,None,G,None,Y,None,1991.0,D/S,M,None,None,3.736796e+09,00296,F1
2,None,T,O,None,M,1961.0,09302016,M,None,OS,6.666432e+08,93,B2
3,None,O,O,None,M,1988.0,09302016,None,None,AA,9.246846e+10,00199,B2
4,None,O,O,None,M,2012.0,09302016,None,None,AA,9.246846e+10,00199,B2


### Explore Global Temperature data

In [10]:
global_temp_df.limit(5).toPandas()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [11]:
global_temp_df.select("Country").dropDuplicates().sort("Country").limit(20).toPandas()

,Country
0,Afghanistan
1,Albania
2,Algeria
3,Angola
4,Argentina
5,Armenia
6,Australia
7,Austria
8,Azerbaijan
9,Bahamas


In [12]:
# global_temp_df.filter(global_temp_df["Country"] == 'United States').sort("dt").limit(5).toPandas()

### Explore US Cities data

In [13]:
us_cities_df.filter(us_cities_df["State"] == 'California').sort("City").limit(10).toPandas()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Alameda,California,41.4,37747,40867,78614,4504,18841,2.52,CA,White,44232
1,Alameda,California,41.4,37747,40867,78614,4504,18841,2.52,CA,American Indian and Alaska Native,1329
2,Alameda,California,41.4,37747,40867,78614,4504,18841,2.52,CA,Black or African-American,7364
3,Alameda,California,41.4,37747,40867,78614,4504,18841,2.52,CA,Hispanic or Latino,8265
4,Alameda,California,41.4,37747,40867,78614,4504,18841,2.52,CA,Asian,27984
5,Alhambra,California,41.0,42184,43388,85572,1673,44441,2.89,CA,Asian,44067
6,Alhambra,California,41.0,42184,43388,85572,1673,44441,2.89,CA,American Indian and Alaska Native,687
7,Alhambra,California,41.0,42184,43388,85572,1673,44441,2.89,CA,White,20811
8,Alhambra,California,41.0,42184,43388,85572,1673,44441,2.89,CA,Hispanic or Latino,31386
9,Alhambra,California,41.0,42184,43388,85572,1673,44441,2.89,CA,Black or African-American,1905


### Explore Airport Code data

In [14]:
airport_code_df.filter(airport_code_df["iso_country"] == 'US').limit(5).toPandas()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11,NA,US,US-PA,Bensalem,00A,None,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435,NA,US,US-KS,Leoti,00AA,None,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450,NA,US,US-AK,Anchor Point,00AK,None,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820,NA,US,US-AL,Harvest,00AL,None,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237,NA,US,US-AR,Newport,None,None,None,"-91.254898, 35.6087"


## Cleaning Steps
Document steps necessary to clean the data

### Clean the SAS data

In [15]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

sas_cleaned_df = sas_df.select(
    sas_df.admnum.cast(IntegerType()).cast(StringType()),
    sas_df.cicid.cast(IntegerType()).cast(StringType()),
    sas_df.insnum.cast(IntegerType()).cast(StringType()),
    sas_df.i94bir.cast(IntegerType()),
    sas_df.biryear.cast(IntegerType()),
    sas_df.gender,
    sas_df.airline,
    sas_df.fltno,
    sas_df.visatype,
    sas_df.arrdate.cast(IntegerType()),
    sas_df.depdate.cast(IntegerType()),
    sas_df.i94yr.cast(IntegerType()),
    sas_df.i94mon.cast(IntegerType()),
    sas_df.i94cit.cast(IntegerType()),
    sas_df.i94res.cast(IntegerType()),
    sas_df.i94port,
    sas_df.i94mode.cast(IntegerType()),
    sas_df.i94addr,
    sas_df.i94visa.cast(IntegerType()).cast(StringType()),
)

sas_cleaned_df.printSchema()
sas_cleaned_df.limit(10).toPandas()

root
 |-- admnum: string (nullable = true)
 |-- cicid: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- i94bir: integer (nullable = true)
 |-- biryear: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- fltno: string (nullable = true)
 |-- visatype: string (nullable = true)
 |-- arrdate: integer (nullable = true)
 |-- depdate: integer (nullable = true)
 |-- i94yr: integer (nullable = true)
 |-- i94mon: integer (nullable = true)
 |-- i94cit: integer (nullable = true)
 |-- i94res: integer (nullable = true)
 |-- i94port: string (nullable = true)
 |-- i94mode: integer (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- i94visa: string (nullable = true)



,admnum,cicid,insnum,i94bir,biryear,gender,airline,fltno,visatype,arrdate,depdate,i94yr,i94mon,i94cit,i94res,i94port,i94mode,i94addr,i94visa
0,1897628485,6,None,37,1979,None,None,None,B2,20573,NaN,2016,4,692,692,XXX,NaN,None,2
1,2147483647,7,None,25,1991,M,None,00296,F1,20551,NaN,2016,4,254,276,ATL,1.0,AL,3
2,666643185,15,None,55,1961,M,OS,93,B2,20545,20691.0,2016,4,101,101,WAS,1.0,MI,2
3,2147483647,16,None,28,1988,None,AA,00199,B2,20545,20567.0,2016,4,101,101,NYC,1.0,MA,2
4,2147483647,17,None,4,2012,None,AA,00199,B2,20545,20567.0,2016,4,101,101,NYC,1.0,MA,2
5,2147483647,18,None,57,1959,None,AZ,00602,B1,20545,20555.0,2016,4,101,101,NYC,1.0,MI,1
6,2147483647,19,None,63,1953,None,AZ,00602,B2,20545,20558.0,2016,4,101,101,NYC,1.0,NJ,2
7,2147483647,20,None,57,1959,None,AZ,00602,B2,20545,20558.0,2016,4,101,101,NYC,1.0,NJ,2
8,2147483647,21,None,46,1970,None,AZ,00602,B2,20545,20553.0,2016,4,101,101,NYC,1.0,NY,2
9,2147483647,22,None,48,1968,None,AZ,00608,B1,20545,20562.0,2016,4,101,101,NYC,1.0,NY,1


### Create 5 new datasets based on `I94_SAS_Labels_Descriptions.SAS`

- I94 SAS dataset has a number of labels must be defined in `I94_SAS_Labels_Descriptions.SAS`, thus it must be run before in SAS system to define the lables for the dataset.

- However, we cannot do that in our code, thus we have to create some datasets based on the label description to match it with the data we extracted.

#### 1. Create dataset for I94 countries

In [16]:
i94_country_df = spark.read \
    .option("delimiter", ";") \
    .option("header", True) \
    .option("inferSchema", True) \
    .csv('label_i94cntyl.csv')

i94_country_df = i94_country_df \
    .withColumn("country", translate("country", "'", "")) \
    .withColumn("code", i94_country_df.code.cast(IntegerType()))

i94_country_df.printSchema()
i94_country_df.limit(10).toPandas()

root
 |-- code: integer (nullable = true)
 |-- country: string (nullable = true)



,code,country
0,582,"MEXICO Air Sea, and Not Reported (I-94, no l..."
1,236,AFGHANISTAN
2,101,ALBANIA
3,316,ALGERIA
4,102,ANDORRA
5,324,ANGOLA
6,529,ANGUILLA
7,518,ANTIGUA-BARBUDA
8,687,ARGENTINA
9,151,ARMENIA


#### 2. Create dataset for I94 ports

In [17]:
i94_port_df = spark.read \
    .option("delimiter", ";") \
    .option("header", True) \
    .option("inferSchema", True) \
    .csv('label_i94prtl.csv')

i94_port_df = i94_port_df \
    .withColumn("code", translate("code", "\t", '')) \
    .withColumn("code", translate("code", "'", '')) \
    .withColumn("port", translate("port", "\t", '')) \
    .withColumn("port", translate("port", "'", ''))

i94_port_df = i94_port_df \
    .withColumn("code", trim(i94_port_df.code)) \
    .withColumn("port", trim(i94_port_df.port))

i94_port_df.printSchema()
i94_port_df.limit(10).toPandas()

root
 |-- code: string (nullable = true)
 |-- port: string (nullable = true)



,code,port
0,ALC,"ALCAN, AK"
1,ANC,"ANCHORAGE, AK"
2,BAR,"BAKER AAF - BAKER ISLAND, AK"
3,DAC,"DALTONS CACHE, AK"
4,PIZ,"DEW STATION PT LAY DEW, AK"
5,DTH,"DUTCH HARBOR, AK"
6,EGL,"EAGLE, AK"
7,FRB,"FAIRBANKS, AK"
8,HOM,"HOMER, AK"
9,HYD,"HYDER, AK"


#### 3. Create dataset for I94 modes

In [18]:
i94_mode_df = spark.read \
    .option("delimiter", ";") \
    .option("header", True) \
    .option("inferSchema", True) \
    .csv('label_i94model.csv')

i94_mode_df = i94_mode_df \
    .withColumn("mode", trim(i94_mode_df.mode)) \
    .withColumn("mode", translate("mode", "'", "")) \
    .withColumn("code", i94_mode_df.code.cast(IntegerType()))

i94_mode_df.printSchema()
i94_mode_df.limit(5).toPandas()

root
 |-- code: integer (nullable = true)
 |-- mode: string (nullable = true)



,code,mode
0,1,Air
1,2,Sea
2,3,Land
3,9,Not reported


#### 4. Create dataset for I94 states

In [19]:
i94_addr_df = spark.read \
    .option("delimiter", ";") \
    .option("header", True) \
    .option("inferSchema", True) \
    .csv('label_i94addrl.csv')

i94_addr_df = i94_addr_df \
    .withColumn("code", trim(i94_addr_df.code)) \
    .withColumn("code", translate("code", "\t", '')) \
    .withColumn("code", translate("code", "'", "")) \
    .withColumn("state", translate("state", "'", ""))

i94_addr_df.printSchema()
i94_addr_df.limit(5).toPandas()

root
 |-- code: string (nullable = true)
 |-- state: string (nullable = true)



,code,state
0,AL,ALABAMA
1,AK,ALASKA
2,AZ,ARIZONA
3,AR,ARKANSAS
4,CA,CALIFORNIA


#### 5. Create dataset for I94 visa types

In [20]:
i94_visa_df = spark.read \
    .option("delimiter", ";") \
    .option("header", True) \
    .option("inferSchema", True) \
    .csv('label_i94visa.csv')

i94_visa_df = i94_visa_df \
    .withColumn("code", trim(i94_visa_df.code)) \
    .withColumn("category", trim(i94_visa_df.category)) 

i94_visa_df = i94_visa_df.withColumn("code", i94_visa_df.code.cast(IntegerType()))

i94_visa_df.printSchema()
i94_visa_df.limit(5).toPandas()

root
 |-- code: integer (nullable = true)
 |-- category: string (nullable = true)



,code,category
0,1,Business
1,2,Pleasure
2,3,Student


---

# Step 3: Define the Data Model
## 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

![data-model](data-model.png)

This model is an OLAP Snowflake schema, with 3 fact tables: 
- `fact_immigration`
- `fact_global_temp` 
- `fact_us_cities`

between other dimensional tables, such as:
- `dim_us_cities_race`
- `dim_country`
- `dim_airport_code`
- `dim_port`
- `dim_mode`
- `dim_state`
- `dim_visa`

It is to make it easier to create meaningful data, as we all can see the **Immigration data** is the main dataset which we want to explore & identify metrics based on it. The other dimensional tables is to create attributes or definitions to what we have in the immigration data. 

The **US Cities data** & **Global Temperature data** also contains some metrics which we want to aggregate later, thus making it as a fact table will do.

## 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

- Create Fact tables    
1. Create `fact_immigration` table using I94 SAS data
2. Create `fact_global_temp` table using World Temperature data
3. Create `fact_us_cities` table using US Cities Demographics data
- Create Big Dim tables
4. Create `dim_us_cities_race` table using US cities Demographics data
5. Create `dim_airport_code` table using Airport Codes data
- Create Small Dim tables
6. Create `dim_country` table
7. Create `dim_port` table 
8. Create `dim_mode` table
9. Create `dim_state` table
10. Create `dim_visa` table

---

# Step 4: Run Pipelines to Model the Data 
## 4.1 Create the data model
Build the data pipelines to create the data model.

## Create Fact tables

### 1. Create `fact_immigration` table

In [21]:
fact_immigration_df = sas_cleaned_df.select(
    sas_cleaned_df.admnum.alias("admission_number"),
    sas_cleaned_df.cicid.alias("cic_id"),
    sas_cleaned_df.insnum.alias("insurance_number"),
    sas_cleaned_df.biryear.alias("birth_year"),
    sas_cleaned_df.i94bir.alias("age"),
    sas_cleaned_df.gender,
    sas_cleaned_df.airline,
    sas_cleaned_df.fltno.alias("flight_number"),
    sas_cleaned_df.visatype,
    sas_cleaned_df.arrdate.alias("arrival_date"),
    sas_cleaned_df.depdate.alias("departure_date"),
    sas_cleaned_df.i94cit.alias("origin_country_code"),
    sas_cleaned_df.i94res.alias("residential_country_code"),
    sas_cleaned_df.i94port.alias("port_code"),
    sas_cleaned_df.i94mode.alias("mode_code"),
    sas_cleaned_df.i94addr.alias("state_code"),
    sas_cleaned_df.i94visa.alias("visa_code"),
    sas_cleaned_df.i94yr.alias("year"),
    sas_cleaned_df.i94mon.alias("month"),
    )

fact_immigration_df.printSchema()
fact_immigration_df.limit(5).toPandas()

# Write table
fact_immigration_df.write.partitionBy("year", "month").mode("overwrite").parquet("dwh/fact_immigration")

root
 |-- admission_number: string (nullable = true)
 |-- cic_id: string (nullable = true)
 |-- insurance_number: string (nullable = true)
 |-- birth_year: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- flight_number: string (nullable = true)
 |-- visatype: string (nullable = true)
 |-- arrival_date: integer (nullable = true)
 |-- departure_date: integer (nullable = true)
 |-- origin_country_code: integer (nullable = true)
 |-- residential_country_code: integer (nullable = true)
 |-- port_code: string (nullable = true)
 |-- mode_code: integer (nullable = true)
 |-- state_code: string (nullable = true)
 |-- visa_code: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)



### 2. Create `fact_global_temp` table

In [22]:
fact_global_temp_df = global_temp_df \
    .withColumn("year", year(global_temp_df["dt"])) \
    .withColumn("month", month(global_temp_df["dt"])) \
    .withColumn("day", dayofmonth(global_temp_df["dt"]))

fact_global_temp_df.printSchema()
fact_global_temp_df.limit(10).toPandas()

# Write table
fact_global_temp_df.write.partitionBy("year", "month").mode("overwrite").parquet("dwh/fact_global_temp")

root
 |-- dt: timestamp (nullable = true)
 |-- AverageTemperature: double (nullable = true)
 |-- AverageTemperatureUncertainty: double (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)



### 3. Create `fact_us_cities` table

In [23]:
fact_us_cities_df = us_cities_df.select(
    us_cities_df["State Code"].alias("StateCode"),
    us_cities_df["State"],
    us_cities_df["City"],    
    us_cities_df["Median Age"].alias("MedianAge"),
    us_cities_df["Male Population"].alias("MalePopulation"),
    us_cities_df["Female Population"].alias("FemalePopulation"),
    us_cities_df["Total Population"].alias("TotalPopulation"),
    us_cities_df["Number of Veterans"].alias("NumberofVeterans"),
    us_cities_df["Foreign-born"],
    us_cities_df["Average Household Size"].alias("AverageHouseholdSize"),
    ).dropDuplicates()

fact_us_cities_df.printSchema()
fact_us_cities_df.sort(["StateCode", "State", "City"]).limit(10).toPandas()

# Write table
fact_us_cities_df.write.partitionBy("StateCode").mode("overwrite").parquet("dwh/fact_us_cities")

root
 |-- StateCode: string (nullable = true)
 |-- State: string (nullable = true)
 |-- City: string (nullable = true)
 |-- MedianAge: double (nullable = true)
 |-- MalePopulation: integer (nullable = true)
 |-- FemalePopulation: integer (nullable = true)
 |-- TotalPopulation: integer (nullable = true)
 |-- NumberofVeterans: integer (nullable = true)
 |-- Foreign-born: integer (nullable = true)
 |-- AverageHouseholdSize: double (nullable = true)



---

## Create Big Dim tables

### 4. Create `dim_us_cities_race` table

In [24]:
dim_us_cities_race_df = us_cities_df.select(
    us_cities_df["State Code"].alias("StateCode"),
    us_cities_df["State"],
    us_cities_df["City"],
    us_cities_df["Race"],
    us_cities_df["Count"],
)
dim_us_cities_race_df.printSchema()
dim_us_cities_race_df.sort(["StateCode", "State", "City", "Race"]).limit(10).toPandas()

# Write table
dim_us_cities_race_df.write.partitionBy("StateCode").mode("overwrite").parquet("dwh/dim_us_cities_race")

root
 |-- StateCode: string (nullable = true)
 |-- State: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: integer (nullable = true)



### 5. Create `dim_airport_code` table

In [25]:
dim_airport_code_df = airport_code_df.withColumn("iso_state", substring('iso_region', 4, 2))

dim_airport_code_df.printSchema()
dim_airport_code_df.limit(5).toPandas()

# Write table
dim_airport_code_df.write.partitionBy("iso_country", "iso_state").mode("overwrite").parquet("dwh/dim_airport_code")

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: integer (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)
 |-- iso_state: string (nullable = true)



## Create Small Dim tables

### 6. Create `dim_country` table

In [26]:
dim_country_df = i94_country_df.withColumnRenamed("code", "country_code")
dim_country_df.printSchema()
dim_country_df.show(10, truncate=False)

# Write table
dim_country_df.write.mode("overwrite").parquet("dwh/dim_country")

root
 |-- country_code: integer (nullable = true)
 |-- country: string (nullable = true)

+------------+-----------------------------------------------------------+
|country_code|country                                                    |
+------------+-----------------------------------------------------------+
|582         |  MEXICO Air Sea, and Not Reported (I-94, no land arrivals)|
|236         |  AFGHANISTAN                                              |
|101         |  ALBANIA                                                  |
|316         |  ALGERIA                                                  |
|102         |  ANDORRA                                                  |
|324         |  ANGOLA                                                   |
|529         |  ANGUILLA                                                 |
|518         |  ANTIGUA-BARBUDA                                          |
|687         |  ARGENTINA                                                |
|151      

### 7. Create `dim_port` table

In [27]:
dim_port_df = i94_port_df.withColumnRenamed("code", "port_code")

dim_port_df.printSchema()
dim_port_df.show(10, truncate=False)

# Write table
dim_port_df.write.mode("overwrite").parquet("dwh/dim_port")

root
 |-- port_code: string (nullable = true)
 |-- port: string (nullable = true)

+---------+----------------------------+
|port_code|port                        |
+---------+----------------------------+
|ALC      |ALCAN, AK                   |
|ANC      |ANCHORAGE, AK               |
|BAR      |BAKER AAF - BAKER ISLAND, AK|
|DAC      |DALTONS CACHE, AK           |
|PIZ      |DEW STATION PT LAY DEW, AK  |
|DTH      |DUTCH HARBOR, AK            |
|EGL      |EAGLE, AK                   |
|FRB      |FAIRBANKS, AK               |
|HOM      |HOMER, AK                   |
|HYD      |HYDER, AK                   |
+---------+----------------------------+
only showing top 10 rows



### 8. Create `dim_mode` table

In [28]:
dim_mode_df = i94_mode_df.withColumnRenamed("code", "mode_code")

dim_mode_df.printSchema()
dim_mode_df.show()

# Write table
dim_mode_df.write.mode("overwrite").parquet("dwh/dim_mode")

root
 |-- mode_code: integer (nullable = true)
 |-- mode: string (nullable = true)

+---------+------------+
|mode_code|        mode|
+---------+------------+
|        1|         Air|
|        2|         Sea|
|        3|        Land|
|        9|Not reported|
+---------+------------+



### 9. Create `dim_state` table

In [29]:
dim_state_df = i94_addr_df.withColumnRenamed("code", "state_code")
dim_state_df.printSchema()
dim_state_df.show()

# Write table
dim_state_df.write.mode("overwrite").parquet("dwh/dim_state")

root
 |-- state_code: string (nullable = true)
 |-- state: string (nullable = true)

+----------+-----------------+
|state_code|            state|
+----------+-----------------+
|        AL|          ALABAMA|
|        AK|           ALASKA|
|        AZ|          ARIZONA|
|        AR|         ARKANSAS|
|        CA|       CALIFORNIA|
|        CO|         COLORADO|
|        CT|      CONNECTICUT|
|        DE|         DELAWARE|
|        DC|DIST. OF COLUMBIA|
|        FL|          FLORIDA|
|        GA|          GEORGIA|
|        GU|             GUAM|
|        HI|           HAWAII|
|        ID|            IDAHO|
|        IL|         ILLINOIS|
|        IN|          INDIANA|
|        IA|             IOWA|
|        KS|           KANSAS|
|        KY|         KENTUCKY|
|        LA|        LOUISIANA|
+----------+-----------------+
only showing top 20 rows



### 10. Create `dim_visa` table

In [30]:
dim_visa_df = i94_visa_df.withColumnRenamed("code", "visa_code")
dim_visa_df.printSchema()
dim_visa_df.show()

# Write table
dim_visa_df.write.mode("overwrite").parquet("dwh/dim_visa")

root
 |-- visa_code: integer (nullable = true)
 |-- category: string (nullable = true)

+---------+--------+
|visa_code|category|
+---------+--------+
|        1|Business|
|        2|Pleasure|
|        3| Student|
+---------+--------+



## 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

## Source/Count checks

In [31]:
# Perform quality checks here
list_of_tables = [
    fact_immigration_df,
    fact_global_temp_df,
    fact_us_cities_df,
    dim_us_cities_race_df,
    dim_airport_code_df,
    dim_country_df,
    dim_port_df,
    dim_mode_df,
    dim_state_df,
    dim_visa_df,
]
for table in list_of_tables:
    if (table.count() > 0):
        print(f"Table {table} has {table.count()} records") 
    else:
        print(f"There was no records in {table}!")

Table DataFrame[admission_number: string, cic_id: string, insurance_number: string, birth_year: int, age: int, gender: string, airline: string, flight_number: string, visatype: string, arrival_date: int, departure_date: int, origin_country_code: int, residential_country_code: int, port_code: string, mode_code: int, state_code: string, visa_code: string, year: int, month: int] has 3096313 records
Table DataFrame[dt: timestamp, AverageTemperature: double, AverageTemperatureUncertainty: double, City: string, Country: string, Latitude: string, Longitude: string, year: int, month: int, day: int] has 8599212 records
Table DataFrame[StateCode: string, State: string, City: string, MedianAge: double, MalePopulation: int, FemalePopulation: int, TotalPopulation: int, NumberofVeterans: int, Foreign-born: int, AverageHouseholdSize: double] has 596 records
Table DataFrame[StateCode: string, State: string, City: string, Race: string, Count: int] has 2891 records
Table DataFrame[ident: string, type: s

## Integrity constraints

Country Joined

In [32]:
country_joined_test = fact_immigration_df \
    .join(dim_country_df, fact_immigration_df.origin_country_code == dim_country_df.country_code, "left") \
    .drop(dim_country_df["country_code"]) \
    .withColumnRenamed("country", "original_country") \
    .join(dim_country_df, fact_immigration_df.residential_country_code == dim_country_df.country_code, "left") \
    .drop(dim_country_df["country_code"]) \
    .withColumnRenamed("country", "residential_country") 

country_joined_test.printSchema()

country_joined_test = country_joined_test.select(["admission_number", "origin_country_code", "original_country", \
                            "residential_country_code", "residential_country"])
country_joined_test.show(10)
country_joined_test.count()

root
 |-- admission_number: string (nullable = true)
 |-- cic_id: string (nullable = true)
 |-- insurance_number: string (nullable = true)
 |-- birth_year: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- flight_number: string (nullable = true)
 |-- visatype: string (nullable = true)
 |-- arrival_date: integer (nullable = true)
 |-- departure_date: integer (nullable = true)
 |-- origin_country_code: integer (nullable = true)
 |-- residential_country_code: integer (nullable = true)
 |-- port_code: string (nullable = true)
 |-- mode_code: integer (nullable = true)
 |-- state_code: string (nullable = true)
 |-- visa_code: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- original_country: string (nullable = true)
 |-- residential_country: string (nullable = true)

+----------------+-------------------+----------------+-----------------------

3096313

Port Join

In [33]:
port_joined_test = fact_immigration_df \
    .join(dim_port_df, fact_immigration_df.port_code == dim_port_df.port_code, "left") \
    .drop(dim_port_df["port_code"])
port_joined_test.printSchema()
port_joined_test.select(["admission_number", "port"]).show(10)
port_joined_test.count()

root
 |-- admission_number: string (nullable = true)
 |-- cic_id: string (nullable = true)
 |-- insurance_number: string (nullable = true)
 |-- birth_year: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- flight_number: string (nullable = true)
 |-- visatype: string (nullable = true)
 |-- arrival_date: integer (nullable = true)
 |-- departure_date: integer (nullable = true)
 |-- origin_country_code: integer (nullable = true)
 |-- residential_country_code: integer (nullable = true)
 |-- port_code: string (nullable = true)
 |-- mode_code: integer (nullable = true)
 |-- state_code: string (nullable = true)
 |-- visa_code: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- port: string (nullable = true)

+----------------+--------------------+
|admission_number|                port|
+----------------+--------------------+
|      1897628485|NOT

3096313

Mode Join

In [34]:
mode_joined_test = fact_immigration_df \
    .join(dim_mode_df, fact_immigration_df.mode_code == dim_mode_df.mode_code, "left") \
    .drop(dim_mode_df["mode_code"]) \

mode_joined_test.select(["admission_number", "mode_code", "mode"]).show(10)
mode_joined_test.count()

+----------------+---------+----+
|admission_number|mode_code|mode|
+----------------+---------+----+
|      1897628485|     null|null|
|      2147483647|        1| Air|
|       666643185|        1| Air|
|      2147483647|        1| Air|
|      2147483647|        1| Air|
|      2147483647|        1| Air|
|      2147483647|        1| Air|
|      2147483647|        1| Air|
|      2147483647|        1| Air|
|      2147483647|        1| Air|
+----------------+---------+----+
only showing top 10 rows



3096313

State Join

In [35]:
state_joined_test = fact_immigration_df \
    .join(dim_state_df, fact_immigration_df.state_code == dim_state_df.state_code, "left") \
    .drop(dim_state_df["state_code"]) \

state_joined_test.select(["admission_number", "state_code", "state"]).show(10)
state_joined_test.count()

+----------------+----------+-------------+
|admission_number|state_code|        state|
+----------------+----------+-------------+
|      1897628485|      null|         null|
|      2147483647|        AL|      ALABAMA|
|       666643185|        MI|     MICHIGAN|
|      2147483647|        MA|MASSACHUSETTS|
|      2147483647|        MA|MASSACHUSETTS|
|      2147483647|        MI|     MICHIGAN|
|      2147483647|        NJ|   NEW JERSEY|
|      2147483647|        NJ|   NEW JERSEY|
|      2147483647|        NY|     NEW YORK|
|      2147483647|        NY|     NEW YORK|
+----------------+----------+-------------+
only showing top 10 rows



3096313

Visa Join

In [36]:
visa_joined_test = fact_immigration_df \
    .join(dim_visa_df, fact_immigration_df.visa_code == dim_visa_df.visa_code, "left") \
    .drop(dim_visa_df["visa_code"]) 

visa_joined_test.select(["admission_number", "visa_code", "category"]).show(10)
visa_joined_test.count()

+----------------+---------+--------+
|admission_number|visa_code|category|
+----------------+---------+--------+
|      1897628485|        2|Pleasure|
|      2147483647|        3| Student|
|       666643185|        2|Pleasure|
|      2147483647|        2|Pleasure|
|      2147483647|        2|Pleasure|
|      2147483647|        1|Business|
|      2147483647|        2|Pleasure|
|      2147483647|        2|Pleasure|
|      2147483647|        2|Pleasure|
|      2147483647|        1|Business|
+----------------+---------+--------+
only showing top 10 rows



3096313

## Sample Queries

In [37]:
# Create views
fact_immigration_df.createOrReplaceTempView("fact_immigration")
fact_global_temp_df.createOrReplaceTempView("fact_global_temp")
fact_us_cities_df.createOrReplaceTempView("fact_us_cities")

dim_us_cities_race_df.createOrReplaceTempView("dim_us_cities_race")
dim_airport_code_df.createOrReplaceTempView("dim_airport_code")
dim_country_df.createOrReplaceTempView("dim_country")
dim_port_df.createOrReplaceTempView("dim_port")
dim_mode_df.createOrReplaceTempView("dim_mode")
dim_state_df.createOrReplaceTempView("dim_state")
dim_visa_df.createOrReplaceTempView("dim_visa")

### What is the name of the airport of each admission_number?

In [44]:
query = """
        SELECT admission_number, state_code, iso_state, name, type, continent, municipality
        FROM fact_immigration
        LEFT JOIN dim_airport_code ON fact_immigration.state_code = dim_airport_code.iso_state
        """

spark.sql(query).show(5)

+----------------+----------+---------+--------------------+-------------+---------+--------------+
|admission_number|state_code|iso_state|                name|         type|continent|  municipality|
+----------------+----------+---------+--------------------+-------------+---------+--------------+
|      1897628485|      null|     null|                null|         null|     null|          null|
|      2147483647|        AL|       AL|       Lyons Airpark|       closed|       NA|    Rainsville|
|      2147483647|        AL|       AL|       Treeo Airport|small_airport|       NA|        Daphne|
|      2147483647|        AL|       AL|Rocky Springs Air...|small_airport|       NA|        Bremen|
|      2147483647|        AL|       AL|Plant Franklin He...|     heliport|       NA|Smiths Station|
+----------------+----------+---------+--------------------+-------------+---------+--------------+
only showing top 5 rows



### What is the total population of each state in US cities?

In [48]:
query = """
        SELECT StateCode, State, SUM(TotalPopulation) TotalPopulation
        FROM fact_us_cities
        GROUP BY StateCode, State
        """

spark.sql(query).show(5)

+---------+--------------+---------------+
|StateCode|         State|TotalPopulation|
+---------+--------------+---------------+
|       MT|       Montana|         181294|
|       NC|North Carolina|        3060199|
|       MD|      Maryland|        1312129|
|       CO|      Colorado|        2935669|
|       CT|   Connecticut|         885581|
+---------+--------------+---------------+
only showing top 5 rows



### What is the number of admission on each month & year?

In [49]:
query = """
        SELECT year, month, SUM(admission_number) total_admission
        FROM fact_immigration
        GROUP BY year, month
        """

spark.sql(query).show(5)

+----+-----+--------------------+
|year|month|     total_admission|
+----+-----+--------------------+
|2016|    4|6.538320570639435E15|
+----+-----+--------------------+



## 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

### Table: `fact_immigration`
| PK | Field Name | Data Type | Ref | Description |
|---|---|---|---|---|
| x | admission_number | string |  | Admission Number |
|  | cic_id | string |  | CIC Id |
|  | insurance_number | string |  | INS number |
|  | birth_year | integer |  | 4 digit year of birth |
|  | age | integer |  | Age of Respondent in Years |
|  | gender | string |  | Non-immigrant sex |
|  | airline | string |  | Airline used to arrive in U.S. |
|  | flight_number | string |  | Flight number of Airline used to arrive in U.S. |
|  | visatype | string |  | Class of admission legally admitting the non-immigrant to temporarily stay in U.S. |
|  | arrival_date | integer |  | ARRDATE is the Arrival Date in the USA. <br>It is a SAS date numeric field that a permanent format has not been applied. <br>Please apply whichever date format works for you. |
|  | departure_date | integer |  | DEPDATE is the Departure Date from the USA. <br>It is a SAS date numeric field that a permanent format has not been applied. <br>Please apply whichever date format works for you. |
|  | origin_country_code | integer | dim_country | Original country code |
|  | residential_country_code | integer | dim_country | Residential country code |
|  | port_code | string | dim_port | Port code |
|  | mode_code | integer | dim_mode | Mode code |
|  | state_code | string | dim_state | State code |
|  | visa_code | string | dim_visa | Visa code |
|  | year | integer |  | 4 digit year |
|  | month | integer |  | Numeric month |

### Table: `fact_global_temp`
| PK | Field Name | Data Type | Ref | Description |
|---|---|---|---|---|
| x | dt | timestamp |  | Datetime |
|  | AverageTemperature | double |  | Average Temperature in recorded date |
|  | AverageTemperatureUncertainty | double |  | Uncertain Average Temperature in recorded date |
|  | City | string |  | City |
|  | Country | string | dim_country | Country |
|  | Latitude | string |  | Latitude of city |
|  | Longitude | string |  | Longitude of city |
|  | year | integer |  | Year (extracted from dt) |
|  | month | integer |  | Month (extracted from dt) |
|  | day | integer |  | Day (extracted from dt) |

### Table: `fact_us_cities`
| PK | Field Name | Data Type | Ref | Description |
|---|---|---|---|---|
| x | StateCode | string | dim_state, dim_airport_code | State Code |
|  | State | string |  | State name |
|  | City | string |  | City name |
|  | MedianAge | double |  | Median Age of recored city |
|  | MalePopulation | integer |  | Male Population of recorded city |
|  | FemalePopulation | integer |  | Female Population of recorded city |
|  | TotalPopulation | integer |  | Total Population of recorded city |
|  | NumberofVeterans | integer |  | Number of Veterans of recorded city |
|  | Foreign-born | integer |  | Foreign-born of recorded city |
|  | AverageHouseholdSize | double |  | Average Household Size of recorded city |

### Table: `dim_airport_code`
| PK | Field Name | Data Type | Ref | Description |
|---|---|---|---|---|
| x | ident | string |  | Identity code |
|  | type | string |  | Airport type |
|  | name | string |  | Airport name |
|  | elevation_ft | integer |  | Elevation feet |
|  | continent | string |  | Continent of recorded airport |
|  | iso_country | string |  | Country in ISO code |
|  | iso_region | string |  | Region in ISO code |
|  | municipality | string |  | Municipality |
|  | gps_code | string |  | GPS code of recored airport |
|  | iata_code | string |  | IATA code |
|  | local_code | string |  | Local code |
|  | coordinates | string |  | Coordinates of recorded airport |
|  | iso_state | string | fact_immigration | State in ISO code |

### Table: `dim_us_cities_race`
| PK | Field Name | Data Type | Ref | Description |
|---|---|---|---|---|
| x | StateCode | string | fact_us_cities | State Code |
|  | State | string |  | State Name |
|  | City | string |  | City |
|  | Race | string |  | Race |
|  | Count | integer |  | Number of people |

### Table: `dim_country`
| PK | Field Name | Data Type | Ref | Description |
|---|---|---|---|---|
| x | country_code | string |  | Country code |
|  | country | string |  | Country name |

### Table: `dim_port`
| PK | Field Name | Data Type | Ref | Description |
|---|---|---|---|---|
| x | port_code | integer |  | Port code |
|  | port | string |  | Port name |

### Table: `dim_mode`
| PK | Field Name | Data Type | Ref | Description |
|---|---|---|---|---|
| x | mode_code | integer |  | Mode code |
|  | mode | string |  | Travel mode |

### Table: `dim_state`
| PK | Field Name | Data Type | Ref | Description |
|---|---|---|---|---|
| x | state_code | string |  | State code |
|  | state | string |  | State name |

### Table: `dim_visa`
| PK | Field Name | Data Type | Ref | Description |
|---|---|---|---|---|
| x | visa_code | string |  | Visa code |
|  | category | string |  | Visa category |

---

# Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

### The choice of tools and technologies for this project
- Data lake concepts with the usage of:
    - ELT pipelines using Spark (pySpark)
    - Distributed File System technology (HDFS, AWS S3, etc.)
    - OLAP data model for SQL schema-on-read technology (Hive, Athena, Trino, etc.)
- Reason:
    - The data is mostly stored in relational form.
    - Howver, due to the high volume & strange data format of I94 SAS data, it would be better if we store it in data lake rather than traditional data warehouse.
    - We can still use data warehouse technology (such as Redshift) later to implement a DWH, which could be identified as a lakehouse.
#### Who would use this data model:
- We think of different kinds of roles who would make use of this data, but they can be divided into 3 main roles:
    - **Business Analyst**: People with roles relating to this business (in this case, airline industry & relating stakeholders such as immigration officers, governance, etc.). They would want to understand the pattern of immigration & make decision based on the data.
    - **Data Analyst**: People with roles that needs to dive in & explore the data. They would want to create reports, dashboards or visualizations which can help the other stakeholders have better understandings about the data. They can also recognize & do analytics on the data to provide a better insight for businesses.
    - **Data Scientist**: This is one of the reason why we choose data lake. The data which is collected could be used for data scientist roles to enrich, utilize & enhance some of data science projects such as Machine Learning, Deep Learning or etc. as it would be easier for DS to use it in data lake rather than traditional data warehouse.
    
#### What are the different types of questions that this data model can answer
- Some of the question can be:
    1. How many people of each race immigrate to US in a period of time?
    2. What is the percentage of people immigrate to US from a specific country (e.g. Norway)?
    3. What is the pattern/trend of immigration in a specific year? For example, which month that the immigration number is highest comparing to the lowest month?
    

### The frequency of data
- This data should be updated on a daily basis. We can utilize Apache Airflow for such task
    - However, for the dimensional data, they can be updated on a longer basis (weekly-monthly) to reduce the maintenance.

### How we would approach the problem if
- The data was increase by 100x:
    - We will divide the pipeline to months and have to ulitize Apache Airflow to divide it into multile DAGs & tasks.
- The data populates a dashboard that must be updated on a daily basis by 7am every day
    - We will put the data which should be on the dashboard to Amazon Redshift & make the dashboard directly query from it.
- The database needed to be accessed by 100+ people
    - The Amazone Redshift will do the works of the data warehousing. Also, we have to create multiple AWS IAM roles for users to access the Redshift & restricting the number of commands they can query.